<a href="https://colab.research.google.com/github/ruperty/colab/blob/master/basic_cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cartpole with PCT

A demonstration of Perceptual Control Theory applied to the Inverted Pendulum problem using the simulated environment provided by OpenAI gym. 

![alt text](http://www.perceptualrobots.com/wp-content/uploads/2020/05/poster-2.jpg)

[OpenAI](https://openai.com/) is a program for testing the Reinforcement Learning approach to learning.

However, RL is a very convoluted way of addressing this particular problem. RL, and convential AI, seem to see any problem as one of trying to map problem states to actions, which is a fundamental misunderstanding of the nature of the behaviour of living systems. 

PCT provides a very parsimonious and effective approach which avoids any need to map states and actions. Instead action values are varied dynamically in order to control the perceived inputs.

For more information on PCT see http://www.pctweb.org/ and http://perceptualrobots.com/.

# Colab Notebook
This page is a "**notebook**" which allows users to run code in cells online and see the results, without having to set up a programming environment. 

You can even add your own code and run that. Though the file is read-only so if you want to keep it you will have to save it as a copy.

The first few cells are to set up the environment, install and import modules. To run each cell do **shift+enter** or run mutiple cells with the above menu item, **Runtime**.

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install x11-utils  > /dev/null 2>&1

In [0]:
# imports gym module from OpenAI
import gym

# some other required modules
import numpy as np
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from gym.wrappers import Monitor



In [0]:
# Utility functions to enable video recording of gym environment and displaying it
# To enable video, just do "env = wrap_env(env)""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay                 
                 controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start() 

In [0]:
# a function to help with plotting the data
def add_plot_data(ctr, state, action, plot_data):
  if ctr<0:
    plot_data = [[0],[state[0]],[state[1]],[state[2]],[state[3]], [0]]
  else:
   plot_data[0].append(ctr+1)
   plot_data[1].append(state[0])
   plot_data[2].append(state[1])
   plot_data[3].append(state[2])
   plot_data[4].append(state[3])
   if action : dir=1 
   else: dir=-1
   plot_data[5].append(dir)
  return plot_data


# Cartpole Environment

The next cell creates the Cartpole environment and runs some random actions in a loop.

The environment provides the values for the Cart Position, Cart Velocity,	Pole Angle and Pole Velocity At Tip, and takes an action of 0 or 1, to move the cart to the left and right, respectively.

The loop will run until there is a failure such as pole angle exceeding 12 degrees. A "reward" of 1 is awarded for each iteration that the cartpole stays up.

A video will be generated showing the cartpole being run. It will be very short as the system will fail quickly as the actions are random.

In [0]:
env = wrap_env(gym.make("CartPole-v1"))
state = env.reset()
loops = 100
plot_data = add_plot_data(-1, state, None, None)
reward = 0

for i in range(loops):
   env.render()
   action = env.action_space.sample() # take a random action
   observation =env.step(action) 
   plot_data = add_plot_data(i, observation[0], action, plot_data)
   done = observation[2]
   reward+=observation[1]
   if done: 
        if i < loops :
          print("Failed after %d loops with a pole angle of %2.2f degrees" %( i+1, np.rad2deg(observation[0][2])))
        else:
          print("Finished after ", i, " loops")
        env.reset()
        break

print("Reward ", reward, " from ", i+1, " loops")
env.close()
show_video()


Failed after 16 loops with a pole angle of -13.19 degrees
Reward  16.0  from  16  loops


If the video showed very little you can run the above cell again to see if you get a better result.

Ok, let's plot the observed values of the cartpole over the loop.

Here are some from the above run:


*   The angle of the pole.
*   The other values.
*   The actions taken, where -1 is left and 1 is right.

The pole angle quickly exceeds 12 degrees.


In [0]:
import plotly.graph_objects as go

anglefig = go.Figure()
anglefig.add_trace(go.Scatter(mode='markers+lines',x=plot_data[0], y=np.rad2deg(plot_data[3]), name="poleAng"))
anglefig.update_layout(title="Pole Angle", xaxis_title="Time", yaxis_title="Angle (degrees)")
anglefig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[1], name="cartPos"))
fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[2], name="cartVel"))
fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[4], name="poleVel"))
fig.update_layout(title="Other values", xaxis_title="Time", yaxis_title="Angle/Pos")
fig.show()

actionfig = go.Figure()
actionfig.add_trace(go.Scatter(mode='markers+lines',line=dict(color='firebrick'),x=plot_data[0], y=plot_data[5], name="direction"))
actionfig.update_layout(title="Action", xaxis_title="Time", yaxis_title="Direction")
actionfig.show()

# Perceptual Control

Controlling the cartpole with PCT requires a small hierarchy of four levels:



*   At the top, a unit that controls the pole angle by varying the pole velocity,
*   then one that controls pole velocity by varying the cart position,
*   then one that controls cart position by varying the cart velocity,
*   then one that controls cart velocity by varying the direction of movement.

For more details see [Richard Kennaway's description](http://www.livingcontrolsystems.com/demos/inverted_pendulum/inverted_pendulum.pdf). The control model on this page doesn't inlcude an integrator function as the intention is just to keep the cartpole upright in the central position. My next demo will show an integrator allows you to move the cartpole to different postions. 



In [0]:
# a function to help with plotting the data
def add_control_plot_data(ctr, pole_angle_ref, pole_angle, pole_velocity_ref, pole_velocity, cart_position_ref, cart_position, cart_velocity_ref, cart_velocity, action, plot_data):
  if ctr<0:
    plot_data = [[0],[0],[pole_angle],[0],[pole_velocity],[0],[cart_position],[0],[cart_velocity], [0]]
  else:
   plot_data[0].append(ctr+1)
   plot_data[1].append(pole_angle_ref)
   plot_data[2].append(pole_angle)
   plot_data[3].append(pole_velocity_ref)
   plot_data[4].append(pole_velocity)
   plot_data[5].append(cart_position_ref)
   plot_data[6].append(cart_position)
   plot_data[7].append(cart_velocity_ref)
   plot_data[8].append(cart_velocity)
   if action : dir=1 
   else: dir=-1
   plot_data[9].append(dir)
  return plot_data

In [0]:
# the four-level control hierarchy
def controller(ctr, pole_angle_ref, cart_position, cart_velocity, pole_angle, pole_velocity, plot_data):
    pole_velocity_ref=proportional(pole_angle_ref, pole_angle, pole_angle_gain)
    cart_position_ref=proportional(pole_velocity_ref, pole_velocity, pole_velocity_gain)
    cart_velocity_ref=proportional(cart_position_ref, cart_position, cart_position_gain)
    power=proportional(cart_velocity_ref, cart_velocity, cart_velocity_gain)
    
    action=1
    if power>=0:
        action = 0
    
    plot_data=add_control_plot_data(ctr, pole_angle_ref, pole_angle, pole_velocity_ref, pole_velocity, cart_position_ref, cart_position, cart_velocity_ref, cart_velocity, action, plot_data)
      
    return action, plot_data

def proportional(r, p, g):
    return (r-p)*g

def get_states(state):
    cart_position = state[0]
    cart_velocity = state[1]
    pole_angle = state[2]
    pole_velocity = state[3]
    return cart_position, cart_velocity, pole_angle, pole_velocity 


In [0]:
# some configuration settings
pole_angle_ref = 0
loops = 1000
pole_angle_gain = 3
pole_velocity_gain = 2
cart_position_gain = 1
cart_velocity_gain = 1

In [0]:
genv = gym.make("CartPole-v1")
genv._max_episode_steps = 4000
env = wrap_env(genv)
reward = 0

state = env.reset()
cart_position, cart_velocity, pole_angle, pole_velocity = get_states(state)
plot_data=add_control_plot_data(-1, 0, pole_angle, 0, pole_velocity, 0, cart_position, 0, cart_velocity, 0, None)

for ctr in range(loops):
    env.render()
    action, plot_data = controller(ctr, pole_angle_ref, cart_position, cart_velocity, pole_angle, pole_velocity, plot_data)    
    observation =env.step(action)    
    done = observation[2]
    reward+=observation[1]
    cart_position, cart_velocity, pole_angle, pole_velocity = get_states(observation[0])
    if done: 
        print(ctr, observation)
        break
 
print("Reward ", reward, " from ", ctr+1, " loops")    

env.close()
show_video()

Reward  1000.0  from  1000  loops


# Results
It might look, in the above video, that nothing is happening. Well, that is because the control is very good. The system moves the pole to being balanced in a few moves and it stays that way for the remainder of the run. 

As you will see from the plots below each of the perceived (observed) variables denothing the state of the cart are kept low, around zero, which corresponds to the pole being upright and the car in the centre of the track.

The pole angle, particularly, is kept under 0.5 of a degree from vertical, well away from the failure value of 12 degrees.

In [0]:
# the control plots
pole_angle_fig = go.Figure()
pole_angle_fig.add_trace(go.Scatter(x=plot_data[0], y=np.rad2deg(plot_data[1]), name="ref"))
pole_angle_fig.add_trace(go.Scatter(x=plot_data[0], y=np.rad2deg(plot_data[2]), name="perc"))
pole_angle_fig.update_layout(title="Pole Angle", xaxis_title="Time", yaxis_title="Angle (degrees)")
pole_angle_fig.show()

pole_velocity_fig = go.Figure()
pole_velocity_fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[3], name="ref"))
pole_velocity_fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[4], name="perc"))
pole_velocity_fig.update_layout(title="Pole Velocity", xaxis_title="Time", yaxis_title="Velocity")
pole_velocity_fig.show()

cart_position_fig = go.Figure()
cart_position_fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[5], name="ref"))
cart_position_fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[6], name="perc"))
cart_position_fig.update_layout(title="Cart Position", xaxis_title="Time", yaxis_title="Position")
cart_position_fig.show()

cart_velocity_fig = go.Figure()
cart_velocity_fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[7], name="ref"))
cart_velocity_fig.add_trace(go.Scatter(x=plot_data[0], y=plot_data[8], name="perc"))
cart_velocity_fig.update_layout(title="Cart Velocity", xaxis_title="Time", yaxis_title="Velocity")
cart_velocity_fig.show()

actionfig = go.Figure()
actionfig.add_trace(go.Scatter(line=dict(color='firebrick'),x=plot_data[0], y=plot_data[9], name="direction"))
actionfig.update_layout(title="Action", xaxis_title="Time", yaxis_title="Direction")
actionfig.show()



# What Next?

That's it for now. So, what next?

Well, this system controls well, balancing the pole, but it just stays in the same position. It has to do this because its aim is to keep all the goal values at zero. One of these is the cart position which is the centre of the environment. So, it can not move from this position.

It would be great if the cart could move around, while keeping the pole balanced. That is the subject of the next demo, https://colab.research.google.com/drive/1M9Wd66pf6q96z3jMVchu9USHBTZHzxtK#scrollTo=94rcbb2ReOFj.



After that perhaps one that optimises the gain values of the control units to see if the stability of the system can be increased.